In [ ]:
#6.2.1. 互相关运算
!pip install d2l
import torch
from torch import nn
from d2l import torch as d2l

def corr2d(X, K):
    """计算二维互相关运算。"""
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y

     |████████████████████████████████| 81kB 2.6MB/s 


In [ ]:
#的输入张量 X 和卷积核张量 K ，我们来验证一下上述二维互相关运算的输出。
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

In [ ]:
#6.2.2. 卷积层
#基于上面定义的 corr2d 函数实现二维卷积层。在 __init__ 构造函数中，
#将 weight 和 bias 声明为两个模型参数。前向传播函数调用 corr2d 函数并添加偏置。
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias
#高度和宽度分别为 h 和 w的卷积核可以被称为 h×w 卷积或 h×w 卷积核。 
#我们也将带有 h×w 卷积核的卷积层称为 h×w 卷积层

In [ ]:
#6.2.3. 图像中目标的边缘检测
#如下是卷积层的一个简单应用：通过找到像素变化的位置来检测图像中不同颜色的边缘。 
#首先，我们构造一个  6×8  像素的黑白图像。中间四列为黑色（ 0 ），其余像素为白色（ 1 ）。
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [ ]:
#接下来，我们构造一个高度为  1  、宽度为  2  的卷积核 K 。
#当进行互相关运算时，如果水平相邻的两元素相同，则输出为零，否则输出为非零。
K = torch.tensor([[1.0, -1.0]])

In [ ]:
#现在，我们对参数 X （输入）和 K （卷积核）执行互相关运算。 
#如下所示，输出 Y 中的  1  代表从白色到黑色的边缘，  −1  代表从黑色到白色的边缘，其他情况的输出为  0
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [ ]:
#现在我们将输入的二维图像转置，再进行如上的互相关运算。 
#其输出如下，之前检测到的垂直边缘消失了。 不出所料，这个卷积核 K 只可以检测垂直边缘，无法检测水平边缘。
corr2d(X.t(), K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [ ]:
#6.2.4. 学习卷积核
#现在让我们看看是否可以通过仅查看“输入-输出”对来了解由 X 生成 Y 的卷积核。
#我们先构造一个卷积层，并将其卷积核初始化为随机张量。
#接下来，在每次迭代中，我们比较 Y 与卷积层输出的平方误差，然后计算梯度来更新卷积核。
#为了简单起见，我们在此使用内置的二维卷积层，并忽略偏置。
# 构造一个二维卷积层，它具有1个输出通道和形状为（1，2）的卷积核
conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias=False)

# 这个二维卷积层使用四维输入和输出格式（批量大小、通道、高度、宽度），
# 其中批量大小和通道数都为1
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y)**2
    conv2d.zero_grad()
    l.sum().backward()
    # 迭代卷积核
    conv2d.weight.data[:] -= 3e-2 * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'batch {i+1}, loss {l.sum():.3f}')

batch 2, loss 6.556
batch 4, loss 1.230
batch 6, loss 0.260
batch 8, loss 0.066
batch 10, loss 0.020


In [ ]:
#在  10  次迭代之后，误差已经降到足够低。现在我们来看看我们所学的卷积核的权重张量。
conv2d.weight.data.reshape((1, 2))

tensor([[ 1.0000, -0.9745]])